1. 데이터 로드 → 2. 토큰화 → 3. 명사 추출 → 4. N-gram 처리 → 5. 불용어 제거 → 6. 임베딩 모델 생성

📋 단계별 상세 분석

1단계: 데이터 로드 및 기본 전처리
- Ntoken_review.csv 파일 로드
- 필요한 컬럼만 선택 (product_name, rating, skin_type, review, tokens)
- tokens 컬럼을 문자열에서 리스트로 변환

2단계: 명사 토큰 추출
- 형태소 분석된 토큰에서 명사(NNG, NNP)만 추출
= 새로운 컬럼 token_noun 생성
3단계: N-gram 처리
- 3-gram 처리: 특정 3단어 조합을 하나의 의미있는 단어로 변환
- 2-gram 처리: 2단어 조합을 하나의 단어로 변환
- 불용어 및 브랜드명 제거

4단계: 피부타입별 데이터 분리
- 복합성 피부 타입 데이터만 추출
- 불용어 및 1글자 단어 제거

5단계: 임베딩 모델 생성
- Word2Vec과 FastText 모델을 다양한 하이퍼파라미터로 학습
- 키워드별 유사 단어 추출 및 빈도수 계산

In [1]:
# 필요한 라이브러리 import
import pandas as pd
import numpy as np
from ast import literal_eval
from gensim.models import Word2Vec, FastText
import collections
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
from typing import List, Dict, Tuple, Any
import logging

warnings.filterwarnings('ignore')

In [2]:
import pandas as pd

df = pd.read_csv('Ntoken_review.csv')

df.head()

,product_name,reviewer_id,skin_type,skin_tone,skin_concern_1,skin_concern_2,rating,review_date,review,tokens,Ntoken_review
0,닥터지 더모이스처 배리어D 인텐스 크림,리리578,건성,봄웜톤,모공,미백,5,2025.07.24,닥터지 더모이스처 배리어 인텐스크림 제가 이 제품을 사용하고 이렇게 오랜기간...,"[('닥터', 'NNP'), ('지', 'NNB'), ('더', 'MAG'), ('...","['닥터', '이스', '리어', '스크림', '제가', '제품', '사용', '기..."
1,닥터지 더모이스처 배리어D 인텐스 크림,김츄츄,건성,겨울쿨톤,각질,아토피,5,2025.07.27,악건성인데 화장하기 전에 무조건 바르는 크림입니다 화장 잘 뜨고 각질 많이 일어나...,"[('악건성', 'NNG'), ('이', 'VCP'), ('ㄴ데', 'EC'), (...","['악건성', '화장', '크림', '화장', '각질', '건성', '정말', '정..."
2,닥터지 더모이스처 배리어D 인텐스 크림,화사한웜코덕,NaN,NaN,NaN,NaN,5,2025.05.31,현직메컵아티스트입니다이거 안맞는 분 한번도 못봤어용 기초할때 선크림 직전에 발라주고...,"[('현직', 'NNG'), ('메', 'NNG'), ('컵', 'NNG'), ('...","['현직', '아티스트', '기초', '선크림', '직전', '흡수', '선크림',..."
3,닥터지 더모이스처 배리어D 인텐스 크림,앙정,건성,봄웜톤,모공,잡티,5,2025.06.22,보습 필요할때 사용하고 있어요가볍게 바르기 좋고 보습을 필요로할때는 듬뿍 발라주고 있어요,"[('보습', 'NNG'), ('필요', 'NNG'), ('하', 'XSV'), (...","['보습', '필요', '사용', '보습', '필요']"
4,닥터지 더모이스처 배리어D 인텐스 크림,mimilike,건성,웜톤,각질,주름,4,2025.07.11,다른 크림들과 비교해서 확실히 가려움에 효과가 있어요 그래서 저는 아토피 올라오는...,"[('다른', 'MM'), ('크림', 'NNG'), ('들', 'XSN'), ('...","['크림', '비교', '가려움', '효과', '아토피', '부분', '세라마이드'..."


In [3]:
# 1. 원하는 컬럼만 지정
columns = ['product_name', 'rating', 'skin_type', 'review', 'tokens']

# 2. 원하는 컬럼만 불러오기
data = pd.read_csv('Ntoken_review.csv', usecols=columns)

# 3. 데이터 확인
print(f"데이터 형태: {data.shape}")
print(f"컬럼명: {list(data.columns)}")
data.head()

데이터 형태: (1094, 5)
컬럼명: ['product_name', 'skin_type', 'rating', 'review', 'tokens']


,product_name,skin_type,rating,review,tokens
0,닥터지 더모이스처 배리어D 인텐스 크림,건성,5,닥터지 더모이스처 배리어 인텐스크림 제가 이 제품을 사용하고 이렇게 오랜기간...,"[('닥터', 'NNP'), ('지', 'NNB'), ('더', 'MAG'), ('..."
1,닥터지 더모이스처 배리어D 인텐스 크림,건성,5,악건성인데 화장하기 전에 무조건 바르는 크림입니다 화장 잘 뜨고 각질 많이 일어나...,"[('악건성', 'NNG'), ('이', 'VCP'), ('ㄴ데', 'EC'), (..."
2,닥터지 더모이스처 배리어D 인텐스 크림,NaN,5,현직메컵아티스트입니다이거 안맞는 분 한번도 못봤어용 기초할때 선크림 직전에 발라주고...,"[('현직', 'NNG'), ('메', 'NNG'), ('컵', 'NNG'), ('..."
3,닥터지 더모이스처 배리어D 인텐스 크림,건성,5,보습 필요할때 사용하고 있어요가볍게 바르기 좋고 보습을 필요로할때는 듬뿍 발라주고 있어요,"[('보습', 'NNG'), ('필요', 'NNG'), ('하', 'XSV'), (..."
4,닥터지 더모이스처 배리어D 인텐스 크림,건성,4,다른 크림들과 비교해서 확실히 가려움에 효과가 있어요 그래서 저는 아토피 올라오는...,"[('다른', 'MM'), ('크림', 'NNG'), ('들', 'XSN'), ('..."


In [4]:
# string -> list 타입 변경
from ast import literal_eval
data['tokens'] = data['tokens'].apply(literal_eval)

In [5]:
data.head()

,product_name,skin_type,rating,review,tokens
0,닥터지 더모이스처 배리어D 인텐스 크림,건성,5,닥터지 더모이스처 배리어 인텐스크림 제가 이 제품을 사용하고 이렇게 오랜기간...,"[(닥터, NNP), (지, NNB), (더, MAG), (모, MM), (이스, ..."
1,닥터지 더모이스처 배리어D 인텐스 크림,건성,5,악건성인데 화장하기 전에 무조건 바르는 크림입니다 화장 잘 뜨고 각질 많이 일어나...,"[(악건성, NNG), (이, VCP), (ㄴ데, EC), (화장, NNG), (하..."
2,닥터지 더모이스처 배리어D 인텐스 크림,NaN,5,현직메컵아티스트입니다이거 안맞는 분 한번도 못봤어용 기초할때 선크림 직전에 발라주고...,"[(현직, NNG), (메, NNG), (컵, NNG), (아티스트, NNP), (..."
3,닥터지 더모이스처 배리어D 인텐스 크림,건성,5,보습 필요할때 사용하고 있어요가볍게 바르기 좋고 보습을 필요로할때는 듬뿍 발라주고 있어요,"[(보습, NNG), (필요, NNG), (하, XSV), (ㄹ, ETM), (때,..."
4,닥터지 더모이스처 배리어D 인텐스 크림,건성,4,다른 크림들과 비교해서 확실히 가려움에 효과가 있어요 그래서 저는 아토피 올라오는...,"[(다른, MM), (크림, NNG), (들, XSN), (과, JC), (비교, ..."


In [6]:
# 명사 토큰만 분리 
token_noun = []

for sent in data['tokens']:
    sent_noun = []

    for word, tag in sent:
        if tag=='NNG' or tag=='NNP' :
            sent_noun.append(word)

    token_noun.append(sent_noun)

len(token_noun)

1094

In [7]:
data['token_noun'] = token_noun
data.head()

,product_name,skin_type,rating,review,tokens,token_noun
0,닥터지 더모이스처 배리어D 인텐스 크림,건성,5,닥터지 더모이스처 배리어 인텐스크림 제가 이 제품을 사용하고 이렇게 오랜기간...,"[(닥터, NNP), (지, NNB), (더, MAG), (모, MM), (이스, ...","[닥터, 이스, 처, 배, 리어, 인, 텐, 스크림, 제가, 제품, 사용, 기간, ..."
1,닥터지 더모이스처 배리어D 인텐스 크림,건성,5,악건성인데 화장하기 전에 무조건 바르는 크림입니다 화장 잘 뜨고 각질 많이 일어나...,"[(악건성, NNG), (이, VCP), (ㄴ데, EC), (화장, NNG), (하...","[악건성, 화장, 전, 크림, 화장, 각질, 건성, 정말, 정말, 지성, 복합성]"
2,닥터지 더모이스처 배리어D 인텐스 크림,NaN,5,현직메컵아티스트입니다이거 안맞는 분 한번도 못봤어용 기초할때 선크림 직전에 발라주고...,"[(현직, NNG), (메, NNG), (컵, NNG), (아티스트, NNP), (...","[현직, 메, 컵, 아티스트, 기초, 때, 선크림, 직전, 흡수, 선크림, 피부, 자극]"
3,닥터지 더모이스처 배리어D 인텐스 크림,건성,5,보습 필요할때 사용하고 있어요가볍게 바르기 좋고 보습을 필요로할때는 듬뿍 발라주고 있어요,"[(보습, NNG), (필요, NNG), (하, XSV), (ㄹ, ETM), (때,...","[보습, 필요, 때, 사용, 보습, 필요, 때]"
4,닥터지 더모이스처 배리어D 인텐스 크림,건성,4,다른 크림들과 비교해서 확실히 가려움에 효과가 있어요 그래서 저는 아토피 올라오는...,"[(다른, MM), (크림, NNG), (들, XSN), (과, JC), (비교, ...","[크림, 비교, 가려움, 효과, 아토피, 부분, 세라마이드, 크림, 특유, 들보, ..."


In [8]:
def process_trigrams(token_noun):
    """3단어 조합을 처리하는 함수"""
    
    # 변환 규칙 정의
    trigram_rules = {
        "속당김": "속당김",
        "유수분밸런스": "유수분밸런스", 
        "히루론산": "히알루론산",
        "피부진정": "진정",
        "수분부족지성": "수부지",
        "예민피부": "예민",
        "세라마이드": "세라마이드",
        "원플러스원": "원플러스원",
        "피부장벽강화": "피부장벽강화",
        "판테시": "판테놀",
        "모든피부타입": "모든피부타입",
        "힐판테": "판테놀",
        "부승관": "부승관",
        "톤업크림": "톤업크림",
        "쿠링감": "쿨링감",
        "라마이딘": "세라마이딘",
        "피부진정효과": "진정",
        "민감성피부": "민감성",
        "악건성피부": "악건성",
        "지복합성피부": "지복합성",
        "화농성여드름": "화농성여드름"
    }
    
    # 제거할 단어들 ex) 브랜드 
    tri_remove = ['블레미쉬', '차앤박', '리페어크림', '리뉴얼', '아벤느',
                  '리얼베리어', '바이오힐보', '라로슈포', '아토베리어',
                  '바이오더마', '보타닉', '닥터자르카', '베리어익스트림',
                  '자작나무수분크림', '닥터디런', '자르카페어']
    
    processed_sentences = []
    
    for sent in token_noun:
        i = 0
        while i < len(sent) - 2:
            # 3단어 조합 만들기
            trigram = sent[i] + sent[i+1] + sent[i+2]
            
            # 변환 규칙 확인
            if trigram in trigram_rules:
                sent[i] = trigram_rules[trigram]
                del sent[i+1:i+3]  # i+1, i+2 삭제
            # 제거할 단어들 확인
            elif trigram in tri_remove:
                del sent[i:i+3]  # i, i+1, i+2 모두 삭제
            else:
                i += 1  # 매칭되지 않으면 다음으로
        
        processed_sentences.append(sent)
    
    return processed_sentences

# 사용
data['token_noun'] = process_trigrams(token_noun)
data.head()

,product_name,skin_type,rating,review,tokens,token_noun
0,닥터지 더모이스처 배리어D 인텐스 크림,건성,5,닥터지 더모이스처 배리어 인텐스크림 제가 이 제품을 사용하고 이렇게 오랜기간...,"[(닥터, NNP), (지, NNB), (더, MAG), (모, MM), (이스, ...","[닥터, 이스, 처, 배, 리어, 인, 텐, 스크림, 제가, 제품, 사용, 기간, ..."
1,닥터지 더모이스처 배리어D 인텐스 크림,건성,5,악건성인데 화장하기 전에 무조건 바르는 크림입니다 화장 잘 뜨고 각질 많이 일어나...,"[(악건성, NNG), (이, VCP), (ㄴ데, EC), (화장, NNG), (하...","[악건성, 화장, 전, 크림, 화장, 각질, 건성, 정말, 정말, 지성, 복합성]"
2,닥터지 더모이스처 배리어D 인텐스 크림,NaN,5,현직메컵아티스트입니다이거 안맞는 분 한번도 못봤어용 기초할때 선크림 직전에 발라주고...,"[(현직, NNG), (메, NNG), (컵, NNG), (아티스트, NNP), (...","[현직, 메, 컵, 아티스트, 기초, 때, 선크림, 직전, 흡수, 선크림, 피부, 자극]"
3,닥터지 더모이스처 배리어D 인텐스 크림,건성,5,보습 필요할때 사용하고 있어요가볍게 바르기 좋고 보습을 필요로할때는 듬뿍 발라주고 있어요,"[(보습, NNG), (필요, NNG), (하, XSV), (ㄹ, ETM), (때,...","[보습, 필요, 때, 사용, 보습, 필요, 때]"
4,닥터지 더모이스처 배리어D 인텐스 크림,건성,4,다른 크림들과 비교해서 확실히 가려움에 효과가 있어요 그래서 저는 아토피 올라오는...,"[(다른, MM), (크림, NNG), (들, XSN), (과, JC), (비교, ...","[크림, 비교, 가려움, 효과, 아토피, 부분, 세라마이드, 크림, 특유, 들보, ..."


명사만 추출된 리스트를 다루면서 , 의미 있는 2단어 조합(bigram)을 합치거나 불필요한 표현은 제거하는 방식으로 동작한다. 

1. bi_remove : 불용어(쓸모없는 명사) 제거용 리스트
- 이 리스트에 포함된 브랜드명 또는 상품명은 부석 목적에 불필요하므로 삭제 처리
- 문장 내에서 sent[i] + sent[i+1] 이 이 단어들과 일치하면 -> 두 단어 모두 제거

2. Bigram 치환 처리 
if str(sent[i]+sent[i+1]) == "수분크림":
    sent[i] = "수분크림"
    del sent[i+1]
- 예를 들어 명사 리스트가 ['수분', '크림', '촉촉함']이라면 -> 수분 + 크림 을 "수분크림"으로 합쳐서 한 단어로 변환

3. 중복 조건 및 예외 처리
- 일부 조건은 반복되거나 예외 처리한다. 
elif str(sent[i]+sent[i+1]) == "가격대비":
    sent[i] = "가격대비"
    del sent[i+1]
    del sent[i+1]  # 중복 처리
- "가격대비" 같은 경우에는 합성어 이후에 불필요한 단어가 하나 더 붙는 상황을 미리 고려해서 두 번 삭제하는 구조입니다.

4. 합성어 제거 (불용어)
elif sent[i]+sent[i+1] in bi_remove:
    del sent[i]
    del sent[i]
- 예: ['피지오', '겔'] → "피지오겔"은 bi_remove에 포함되어 있으므로 두 단어 모두 제거됩니다.

In [9]:
def process_bigrams(token_noun):
    """2단어 조합을 효율적으로 처리하는 함수"""
    
    # 변환 규칙 정의
    bigram_rules = {
        "수분크림": "수분크림",
        "속건조": "속건조",
        "피부타입": "피부타입",
        "시카": "시카",
        "제품사용": "제품사용",
        "화장솜": "화장솜",
        "유분기": "유분기",
        "수분감": "수분감",
        "스킨케어": "스킨케어",
        "무향": "무향",
        "수부지": "수부지",
        "피부결": "피부결",
        "스킨팩": "스킨팩",
        "티트리": "티트리",
        "당김": "당김",
        "화장전": "화장전",
        "구매의사": "구매의사",
        "좁쌀여드름": "좁쌀여드름",
        "세안후": "세안후",
        "물제형": "물제형",
        "보습감": "보습감",
        "수분부족": "수분부족",
        "악건성": "악건성",
        "사용후": "사용후",
        "보습크림": "보습크림",
        "유수분": "유수분",
        "남자친구": "남자친구",
        "수분충전": "수분충전",
        "유튜브": "유튜브",
        "수분감도": "수분감",
        "피부장벽": "피부장벽",
        "토너팩": "스킨팩",
        "레이어링": "레이어링",
        "선크림": "선크림",
        "가격대비": "가격대비",
        "진정효과": "진정효과",
        "발림성도": "발림성",
        "사용감": "사용감",
        "젤크림": "젤크림",
        "톤업": "톤업",
        "체험단": "체험단",
        "재생크림": "재생크림",
        "쿠링": "쿨링",
        "강추": "강추",
        "배송": "배송",
        "속당김": "속당김",
        "극건성": "극건성",
        "젤타입": "젤타입"
    }
    
    # 제거할 단어들
    bi_remove = ['에스트라', '올리브영', '피지오', '바이오힐', '독도토너', '멀티밤']
    
    processed_sentences = []
    
    for sent in token_noun:
        i = 0
        while i < len(sent) - 1:
            # 2단어 조합 만들기
            bigram = sent[i] + sent[i+1]
            
            # 변환 규칙 확인
            if bigram in bigram_rules:
                sent[i] = bigram_rules[bigram]
                del sent[i+1]
            # 제거할 단어들 확인
            elif bigram in bi_remove:
                del sent[i:i+2]  # i, i+1 삭제
            else:
                i += 1  # 매칭되지 않으면 다음으로
        
        processed_sentences.append(sent)
    
    return processed_sentences

# 사용
data['token_noun'] = process_bigrams(data['token_noun'])

### gensim word2vec, fasttext 파라미터
size = 워드 벡터의 특징 값, 즉, 임베딩 된 벡터의 차원.
- [50, 100]

window = 컨텐스트 윈도우 크기
- [3,4,5]

min_count = 단어 최소 빈도 수 제한 (빈도가 적은 단어들은 학습하지 않는다.)
- 50

workers = 학습을 위한 프로세스 수 
- default

sg = 0은 CBOW, 1은 Skip-gram.
- [0또는 1]

<복합성 키워드> 

1.보습, 보습력,수분, 수분감 (보습감, 수분감)

2.유분기, 기름 (유분기)

3.트러블, 진정, 여드름, 자극, 재생크림 (피부 진정)

4.가격, 용량 (가성비)

5.주름, 탄력 (주름)

6.미백, 기미, 톤업 (톤업)

In [10]:
# 1글자, 불용어 지움

# 불용어 리스트 _ ranks nl 한국어 불용어 사전 텍스트파일 불러옥
with open('stopwords.txt', 'r', encoding='utf-8') as f:
    lines = f.read().splitlines()
    stopwords = [line.rstrip('\n') for line in lines]


# 불용어, 1글자 제거
clean_n = []
token_n = list(data[data['skin_type']=='복합성']['token_noun'].values)
for i in range(len(token_n)):
    clean_n.append([word for word in token_n[i] if word not in stopwords and len(word)>1])

print(len(data[data['skin_type']=='복합성']))
print(len(token_n))


201
201


## 1. 복합성(word2vec, fasttext)

In [11]:
from gensim.models import Word2Vec

word2vec_bok = []

for size in [50, 100]:
    for window in [3,4,5]:
        for sg in [0,1]:
            model = Word2Vec(sentences=clean_n, vector_size=size, window=window, min_count=50, sg=sg, workers=4, epochs=10)
            dictionary = {"model":model, "size":size, "window":window, "sg":sg}
            word2vec_bok.append(dictionary)

In [12]:
from gensim.models import FastText

fasttext_bok = []

for size in [50, 100]:
    for window in [3,4,5]:
        for sg in [0,1]:
            model = FastText(sentences=clean_n, vector_size=size, window=window, min_count=50, sg=sg, workers=4, epochs=10)
            dictionary = {"model":model, "size":size, "window":window, "sg":sg}
            fasttext_bok.append(dictionary)

측면	     Word2Vec	              FastText
정확도	      높음 (훈련된 단어에 대해)	    높음 (새로운 단어도 처리)
범용성	      제한적	                우수
학습 속도	   빠름	                    상대적으로 느림
메모리 사용량	적음	                 많음

# 새로운 화장품 브랜드명이 등장할 때

# Word2Vec
"닥터지새제품" → KeyError (훈련 데이터에 없음)

# FastText  
"닥터지새제품" → ["닥터", "터지", "지새", "새제", "제품"] 
                → 유사한 패턴의 벡터 생성 가능

In [13]:
import collections

keywords = [['보습', '보습력', '수분', '수분감'],
           ['유분기', '기름'],
           ['트러블', '진정', '여드름', '자극', '재생크림'],
           ['가격', '용량'],
           ['주름', '탄력'],
           ['미백', '기미', '톤업']]

most_similar_wv = []
most_similar_ft = []

def safe_most_similar(model, word, topn=10): # topn=10 : 각 키워드당 상위 10개 유사 단어 추출
    """안전하게 유사 단어 추출"""
    try:
        if word in model.wv:
            return model.wv.most_similar(word, topn=topn)
        else:
            return []  # 단어가 없으면 빈 리스트 반환
    except:
        return []


for topic in keywords: # 6개 카테고리 순회
    for key in topic: # 각 카테고리의 키워드 순회
        similar_wv = [] # Word2Vec 모델에서 추출한 유사 단어 리스트
        similar_ft = [] # FastText 모델에서 추출한 유사 단어 리스트
        
        for i in range(len(word2vec_bok)):
            # Word2Vec 모델에서 안전하게 추출
            model_wv = word2vec_bok[i]['model']
            similar_words = safe_most_similar(model_wv, key)
            similar_wv.extend([word for word, _ in similar_words])
            
            # FastText 모델에서 안전하게 추출
            model_ft = fasttext_bok[i]['model']
            similar_words = safe_most_similar(model_ft, key)
            similar_ft.extend([word for word, _ in similar_words])
        
        # 빈도수 계산
        count_wv = collections.Counter(similar_wv)
        count_ft = collections.Counter(similar_ft)
        most_similar_wv.append(count_wv.most_common())
        most_similar_ft.append(count_ft.most_common())

In [14]:
embedding_wv = pd.DataFrame()
embedding_wv['skin_type'] = ['복합성']*18
embedding_wv['keyword'] = sum(keywords, [])
embedding_wv['most_similar'] = most_similar_wv
embedding_wv

,skin_type,keyword,most_similar
0,복합성,보습,[]
1,복합성,보습력,[]
2,복합성,수분,[]
3,복합성,수분감,[]
4,복합성,유분기,[]
5,복합성,기름,[]
6,복합성,트러블,[]
7,복합성,진정,"[(시카, 12), (크림, 12), (사용, 12), (피부, 12), (느낌, ..."
8,복합성,여드름,[]
9,복합성,자극,[]


In [15]:
embedding_ft = pd.DataFrame()
embedding_ft['skin_type'] = ['복합성']*18
embedding_ft['keyword'] = sum(keywords, [])
embedding_ft['most_similar'] = most_similar_ft
embedding_ft

,skin_type,keyword,most_similar
0,복합성,보습,"[(크림, 12), (시카, 12), (구매, 12), (제품, 12), (느낌, ..."
1,복합성,보습력,"[(구매, 12), (진정, 12), (피부, 12), (제품, 12), (느낌, ..."
2,복합성,수분,"[(수분크림, 12), (느낌, 12), (피부, 12), (사용, 12), (진정..."
3,복합성,수분감,"[(느낌, 12), (제품, 12), (수분크림, 12), (구매, 12), (사용..."
4,복합성,유분기,"[(느낌, 12), (사용, 12), (시카, 12), (수분크림, 12), (피부..."
5,복합성,기름,"[(제품, 12), (진정, 12), (크림, 12), (수분크림, 12), (피부..."
6,복합성,트러블,"[(수분크림, 12), (시카, 12), (느낌, 12), (진정, 12), (피부..."
7,복합성,진정,"[(크림, 12), (시카, 12), (사용, 12), (피부, 12), (제품, ..."
8,복합성,여드름,"[(구매, 12), (피부, 12), (진정, 12), (시카, 12), (사용, ..."
9,복합성,자극,"[(크림, 12), (느낌, 12), (진정, 12), (시카, 12), (사용, ..."


의미: Pandas에서 컬럼의 최대 표시 너비를 제한 없이 설정
- 기본값: 보통 50자 정도로 제한
- -1 설정: 제한 없이 전체 내용 표시
- 효과: 긴 텍스트나 리스트가 잘리지 않고 전체가 표시됨

In [16]:
pd.set_option('display.max_colwidth', None)
embedding_wv

,skin_type,keyword,most_similar
0,복합성,보습,[]
1,복합성,보습력,[]
2,복합성,수분,[]
3,복합성,수분감,[]
4,복합성,유분기,[]
5,복합성,기름,[]
6,복합성,트러블,[]
7,복합성,진정,"[(시카, 12), (크림, 12), (사용, 12), (피부, 12), (느낌, 12), (구매, 12), (제품, 12), (수분크림, 12)]"
8,복합성,여드름,[]
9,복합성,자극,[]


In [17]:
embedding_ft

,skin_type,keyword,most_similar
0,복합성,보습,"[(크림, 12), (시카, 12), (구매, 12), (제품, 12), (느낌, 12), (피부, 12), (사용, 12), (수분크림, 12), (진정, 12)]"
1,복합성,보습력,"[(구매, 12), (진정, 12), (피부, 12), (제품, 12), (느낌, 12), (시카, 12), (크림, 12), (사용, 12), (수분크림, 12)]"
2,복합성,수분,"[(수분크림, 12), (느낌, 12), (피부, 12), (사용, 12), (진정, 12), (제품, 12), (크림, 12), (구매, 12), (시카, 12)]"
3,복합성,수분감,"[(느낌, 12), (제품, 12), (수분크림, 12), (구매, 12), (사용, 12), (피부, 12), (진정, 12), (크림, 12), (시카, 12)]"
4,복합성,유분기,"[(느낌, 12), (사용, 12), (시카, 12), (수분크림, 12), (피부, 12), (크림, 12), (진정, 12), (제품, 12), (구매, 12)]"
5,복합성,기름,"[(제품, 12), (진정, 12), (크림, 12), (수분크림, 12), (피부, 12), (느낌, 12), (사용, 12), (시카, 12), (구매, 12)]"
6,복합성,트러블,"[(수분크림, 12), (시카, 12), (느낌, 12), (진정, 12), (피부, 12), (크림, 12), (사용, 12), (구매, 12), (제품, 12)]"
7,복합성,진정,"[(크림, 12), (시카, 12), (사용, 12), (피부, 12), (제품, 12), (느낌, 12), (수분크림, 12), (구매, 12)]"
8,복합성,여드름,"[(구매, 12), (피부, 12), (진정, 12), (시카, 12), (사용, 12), (느낌, 12), (수분크림, 12), (크림, 12), (제품, 12)]"
9,복합성,자극,"[(크림, 12), (느낌, 12), (진정, 12), (시카, 12), (사용, 12), (피부, 12), (구매, 12), (수분크림, 12), (제품, 12)]"


---

## 2. 건성(word2vec, fasttext)

In [18]:
# 1글자, 불용어 지움

# 불용어 리스트 _ ranks nl 한국어 불용어 사전 텍스트파일 불러옥
with open('stopwords.txt', 'r', encoding='utf-8') as f:
    lines = f.read().splitlines()
    stopwords = [line.rstrip('\n') for line in lines]


# 불용어, 1글자 제거 (건성)
clean_d = []
token_d = list(data[data['skin_type']=='건성']['token_noun'].values)
for i in range(len(token_d)):
    clean_d.append([word for word in token_d[i] if word not in stopwords and len(word)>1])

print(len(data[data['skin_type']=='건성']))
print(len(token_d))

197
197


In [19]:
from gensim.models import Word2Vec

word2vec_dry = []

for size in [50, 100]:
    for window in [3,4,5]:
        for sg in [0,1]:
            model = Word2Vec(sentences=clean_d, vector_size=size, window=window, min_count=50, sg=sg, workers=4, epochs=10)
            dictionary = {"model":model, "size":size, "window":window, "sg":sg}
            word2vec_dry.append(dictionary)

In [20]:
from gensim.models import FastText

fasttext_dry = []

for size in [50, 100]:
    for window in [3,4,5]:
        for sg in [0,1]:
            model = FastText(sentences=clean_d, vector_size=size, window=window, min_count=50, sg=sg, workers=4, epochs=10)
            dictionary = {"model":model, "size":size, "window":window, "sg":sg}
            fasttext_dry.append(dictionary)

In [21]:
import collections

keywords_dry = [['보습', '보습력', '수분', '수분감'],
               ['유분기', '기름'],
               ['트러블', '진정', '여드름', '자극', '재생크림'],
               ['가격', '용량'],
               ['주름', '탄력'],
               ['미백', '기미', '톤업']]

most_similar_wv_dry = []
most_similar_ft_dry = []

def safe_most_similar(model, word, topn=10): # topn=10 : 각 키워드당 상위 10개 유사 단어 추출
    """안전하게 유사 단어 추출"""
    try:
        if word in model.wv:
            return model.wv.most_similar(word, topn=topn)
        else:
            return []  # 단어가 없으면 빈 리스트 반환
    except:
        return []


for topic in keywords_dry: # 6개 카테고리 순회
    for key in topic: # 각 카테고리의 키워드 순회
        similar_wv = [] # Word2Vec 모델에서 추출한 유사 단어 리스트
        similar_ft = [] # FastText 모델에서 추출한 유사 단어 리스트
        
        for i in range(len(word2vec_dry)):
            # Word2Vec 모델에서 안전하게 추출
            model_wv = word2vec_dry[i]['model']
            similar_words = safe_most_similar(model_wv, key)
            similar_wv.extend([word for word, _ in similar_words])
            
            # FastText 모델에서 안전하게 추출
            model_ft = fasttext_dry[i]['model']
            similar_words = safe_most_similar(model_ft, key)
            similar_ft.extend([word for word, _ in similar_words])
        
        # 빈도수 계산
        count_wv = collections.Counter(similar_wv)
        count_ft = collections.Counter(similar_ft)
        most_similar_wv_dry.append(count_wv.most_common())
        most_similar_ft_dry.append(count_ft.most_common())

In [22]:
embedding_wv_dry = pd.DataFrame()
embedding_wv_dry['skin_type'] = ['건성']*18
embedding_wv_dry['keyword'] = sum(keywords_dry, [])
embedding_wv_dry['most_similar'] = most_similar_wv_dry
embedding_wv_dry

,skin_type,keyword,most_similar
0,건성,보습,[]
1,건성,보습력,[]
2,건성,수분,[]
3,건성,수분감,[]
4,건성,유분기,[]
5,건성,기름,[]
6,건성,트러블,[]
7,건성,진정,[]
8,건성,여드름,[]
9,건성,자극,[]


In [23]:
embedding_ft_dry = pd.DataFrame()
embedding_ft_dry['skin_type'] = ['건성']*18
embedding_ft_dry['keyword'] = sum(keywords_dry, [])
embedding_ft_dry['most_similar'] = most_similar_ft_dry
embedding_ft_dry

,skin_type,keyword,most_similar
0,건성,보습,"[(건성, 12), (크림, 12), (구매, 12), (제품, 12), (피부, 12), (느낌, 12), (사용, 12), (건조, 12)]"
1,건성,보습력,"[(건성, 12), (구매, 12), (피부, 12), (느낌, 12), (제품, 12), (크림, 12), (사용, 12), (건조, 12)]"
2,건성,수분,"[(건조, 12), (피부, 12), (느낌, 12), (건성, 12), (사용, 12), (구매, 12), (제품, 12), (크림, 12)]"
3,건성,수분감,"[(느낌, 12), (건조, 12), (제품, 12), (구매, 12), (피부, 12), (사용, 12), (건성, 12), (크림, 12)]"
4,건성,유분기,"[(느낌, 12), (사용, 12), (제품, 12), (건조, 12), (피부, 12), (건성, 12), (크림, 12), (구매, 12)]"
5,건성,기름,"[(느낌, 12), (제품, 12), (피부, 12), (크림, 12), (구매, 12), (사용, 12), (건조, 12), (건성, 12)]"
6,건성,트러블,"[(크림, 12), (느낌, 12), (건조, 12), (피부, 12), (사용, 12), (제품, 12), (구매, 12), (건성, 12)]"
7,건성,진정,"[(제품, 12), (피부, 12), (크림, 12), (구매, 12), (느낌, 12), (사용, 12), (건성, 12), (건조, 12)]"
8,건성,여드름,"[(구매, 12), (느낌, 12), (피부, 12), (크림, 12), (사용, 12), (건조, 12), (건성, 12), (제품, 12)]"
9,건성,자극,"[(느낌, 12), (크림, 12), (피부, 12), (구매, 12), (사용, 12), (건조, 12), (건성, 12), (제품, 12)]"


In [24]:
pd.set_option('display.max_colwidth', None)
embedding_wv_dry

,skin_type,keyword,most_similar
0,건성,보습,[]
1,건성,보습력,[]
2,건성,수분,[]
3,건성,수분감,[]
4,건성,유분기,[]
5,건성,기름,[]
6,건성,트러블,[]
7,건성,진정,[]
8,건성,여드름,[]
9,건성,자극,[]


In [25]:
embedding_ft_dry

,skin_type,keyword,most_similar
0,건성,보습,"[(건성, 12), (크림, 12), (구매, 12), (제품, 12), (피부, 12), (느낌, 12), (사용, 12), (건조, 12)]"
1,건성,보습력,"[(건성, 12), (구매, 12), (피부, 12), (느낌, 12), (제품, 12), (크림, 12), (사용, 12), (건조, 12)]"
2,건성,수분,"[(건조, 12), (피부, 12), (느낌, 12), (건성, 12), (사용, 12), (구매, 12), (제품, 12), (크림, 12)]"
3,건성,수분감,"[(느낌, 12), (건조, 12), (제품, 12), (구매, 12), (피부, 12), (사용, 12), (건성, 12), (크림, 12)]"
4,건성,유분기,"[(느낌, 12), (사용, 12), (제품, 12), (건조, 12), (피부, 12), (건성, 12), (크림, 12), (구매, 12)]"
5,건성,기름,"[(느낌, 12), (제품, 12), (피부, 12), (크림, 12), (구매, 12), (사용, 12), (건조, 12), (건성, 12)]"
6,건성,트러블,"[(크림, 12), (느낌, 12), (건조, 12), (피부, 12), (사용, 12), (제품, 12), (구매, 12), (건성, 12)]"
7,건성,진정,"[(제품, 12), (피부, 12), (크림, 12), (구매, 12), (느낌, 12), (사용, 12), (건성, 12), (건조, 12)]"
8,건성,여드름,"[(구매, 12), (느낌, 12), (피부, 12), (크림, 12), (사용, 12), (건조, 12), (건성, 12), (제품, 12)]"
9,건성,자극,"[(느낌, 12), (크림, 12), (피부, 12), (구매, 12), (사용, 12), (건조, 12), (건성, 12), (제품, 12)]"


---

## 3. 지성 (word2vec, fasttext)

In [26]:
# 1글자, 불용어 지움

# 불용어 리스트 _ ranks nl 한국어 불용어 사전 텍스트파일 불러옥
with open('stopwords.txt', 'r', encoding='utf-8') as f:
    lines = f.read().splitlines()
    stopwords = [line.rstrip('\n') for line in lines]


# 불용어, 1글자 제거 (지성)
clean_o = []
token_o = list(data[data['skin_type']=='지성']['token_noun'].values)
for i in range(len(token_o)):
    clean_o.append([word for word in token_o[i] if word not in stopwords and len(word)>1])

print(len(data[data['skin_type']=='지성']))
print(len(token_o))

78
78


In [27]:
# 1. Word2Vec 모델 학습 (지성)
from gensim.models import Word2Vec

word2vec_oily = []

for size in [50, 100]:
    for window in [3,4,5]:
        for sg in [0,1]:
            model = Word2Vec(sentences=clean_o, vector_size=size, window=window, min_count=50, sg=sg, workers=4, epochs=10)
            dictionary = {"model":model, "size":size, "window":window, "sg":sg}
            word2vec_oily.append(dictionary)

In [28]:
# 2. FastText 모델 학습 (지성)
from gensim.models import FastText

fasttext_oily = []

for size in [50, 100]:
    for window in [3,4,5]:
        for sg in [0,1]:
            model = FastText(sentences=clean_o, vector_size=size, window=window, min_count=50, sg=sg, workers=4, epochs=10)
            dictionary = {"model":model, "size":size, "window":window, "sg":sg}
            fasttext_oily.append(dictionary)

In [29]:
import collections

keywords_oily = [['유분기', '기름', '지성'],
                ['보습', '보습력', '수분', '수분감'],
                ['트러블', '진정', '여드름', '자극', '재생크림'],
                ['가격', '용량'],
                ['주름', '탄력'],
                ['미백', '기미', '톤업']]

most_similar_wv_oily = []
most_similar_ft_oily = []

def safe_most_similar(model, word, topn=10): # topn=10 : 각 키워드당 상위 10개 유사 단어 추출
    """안전하게 유사 단어 추출"""
    try:
        if word in model.wv:
            return model.wv.most_similar(word, topn=topn)
        else:
            return []  # 단어가 없으면 빈 리스트 반환
    except:
        return []


for topic in keywords_oily: # 6개 카테고리 순회
    for key in topic: # 각 카테고리의 키워드 순회
        similar_wv = [] # Word2Vec 모델에서 추출한 유사 단어 리스트
        similar_ft = [] # FastText 모델에서 추출한 유사 단어 리스트
        
        for i in range(len(word2vec_oily)):
            # Word2Vec 모델에서 안전하게 추출
            model_wv = word2vec_oily[i]['model']
            similar_words = safe_most_similar(model_wv, key)
            similar_wv.extend([word for word, _ in similar_words])
            
            # FastText 모델에서 안전하게 추출
            model_ft = fasttext_oily[i]['model']
            similar_words = safe_most_similar(model_ft, key)
            similar_ft.extend([word for word, _ in similar_words])
        
        # 빈도수 계산
        count_wv = collections.Counter(similar_wv)
        count_ft = collections.Counter(similar_ft)
        most_similar_wv_oily.append(count_wv.most_common())
        most_similar_ft_oily.append(count_ft.most_common())

In [30]:
embedding_wv_oily = pd.DataFrame()
embedding_wv_oily['skin_type'] = ['지성']*19
embedding_wv_oily['keyword'] = sum(keywords_oily, [])
embedding_wv_oily['most_similar'] = most_similar_wv_oily
embedding_wv_oily

,skin_type,keyword,most_similar
0,지성,유분기,[]
1,지성,기름,[]
2,지성,지성,[]
3,지성,보습,[]
4,지성,보습력,[]
5,지성,수분,[]
6,지성,수분감,[]
7,지성,트러블,[]
8,지성,진정,[]
9,지성,여드름,[]


In [31]:
embedding_ft_oily = pd.DataFrame()
embedding_ft_oily['skin_type'] = ['지성']*19
embedding_ft_oily['keyword'] = sum(keywords_oily, [])
embedding_ft_oily['most_similar'] = most_similar_ft_oily
embedding_ft_oily

,skin_type,keyword,most_similar
0,지성,유분기,"[(피부, 12)]"
1,지성,기름,"[(피부, 12)]"
2,지성,지성,"[(피부, 12)]"
3,지성,보습,"[(피부, 12)]"
4,지성,보습력,"[(피부, 12)]"
5,지성,수분,"[(피부, 12)]"
6,지성,수분감,"[(피부, 12)]"
7,지성,트러블,"[(피부, 12)]"
8,지성,진정,"[(피부, 12)]"
9,지성,여드름,"[(피부, 12)]"


In [32]:
pd.set_option('display.max_colwidth', None)
embedding_wv_oily

,skin_type,keyword,most_similar
0,지성,유분기,[]
1,지성,기름,[]
2,지성,지성,[]
3,지성,보습,[]
4,지성,보습력,[]
5,지성,수분,[]
6,지성,수분감,[]
7,지성,트러블,[]
8,지성,진정,[]
9,지성,여드름,[]


In [33]:
embedding_ft_oily

,skin_type,keyword,most_similar
0,지성,유분기,"[(피부, 12)]"
1,지성,기름,"[(피부, 12)]"
2,지성,지성,"[(피부, 12)]"
3,지성,보습,"[(피부, 12)]"
4,지성,보습력,"[(피부, 12)]"
5,지성,수분,"[(피부, 12)]"
6,지성,수분감,"[(피부, 12)]"
7,지성,트러블,"[(피부, 12)]"
8,지성,진정,"[(피부, 12)]"
9,지성,여드름,"[(피부, 12)]"


In [34]:
# 개별 파일로 저장
embedding_wv.to_pickle('bok_wv_keywords.pkl')      # 복합성 Word2Vec
embedding_ft.to_pickle('bok_ft_keywords.pkl')      # 복합성 FastText
embedding_wv_dry.to_pickle('geon_wv_keywords.pkl') # 건성 Word2Vec
embedding_ft_dry.to_pickle('geon_ft_keywords.pkl') # 건성 FastText
embedding_wv_oily.to_pickle('ji_wv_keywords.pkl')  # 지성 Word2Vec
embedding_ft_oily.to_pickle('ji_ft_keywords.pkl')  # 지성 FastText

# 통합 파일로 저장
all_embeddings = pd.concat([
    embedding_wv, embedding_ft, 
    embedding_wv_dry, embedding_ft_dry,
    embedding_wv_oily, embedding_ft_oily
], ignore_index=True)

all_embeddings.to_pickle('keywords.pkl')
all_embeddings.to_excel('keywords.xlsx')